In [1]:
import pandas as pd

data = [[1, 'Alice'], [2, 'Bob'], [13, 'John'], [6, 'Alex']]
students = pd.DataFrame(data, columns=['student_id', 'student_name']).astype({'student_id':'Int64', 'student_name':'object'})
data = [['Math'], ['Physics'], ['Programming']]
subjects = pd.DataFrame(data, columns=['subject_name']).astype({'subject_name':'object'})
data = [[1, 'Math'], [1, 'Physics'], [1, 'Programming'], [2, 'Programming'], [1, 'Physics'], [1, 'Math'], [13, 'Math'], [13, 'Programming'], [13, 'Physics'], [2, 'Math'], [1, 'Math']]
examinations = pd.DataFrame(data, columns=['student_id', 'subject_name']).astype({'student_id':'Int64', 'subject_name':'object'})

In [2]:
students

,student_id,student_name
0,1,Alice
1,2,Bob
2,13,John
3,6,Alex


In [3]:
subjects

,subject_name
0,Math
1,Physics
2,Programming


In [4]:
examinations

,student_id,subject_name
0,1,Math
1,1,Physics
2,1,Programming
3,2,Programming
4,1,Physics
5,1,Math
6,13,Math
7,13,Programming
8,13,Physics
9,2,Math


In [5]:
# 创建一个临时列，用于合并
students['key'] = 1
subjects['key'] = 1

# 使用 merge 函数进行合并，得到笛卡尔积
# stusub 保存 students 和 subjects 的笛卡尔积
stusub = pd.merge(students, subjects, on='key').drop('key', axis=1)
stusub

,student_id,student_name,subject_name
0,1,Alice,Math
1,1,Alice,Physics
2,1,Alice,Programming
3,2,Bob,Math
4,2,Bob,Physics
5,2,Bob,Programming
6,13,John,Math
7,13,John,Physics
8,13,John,Programming
9,6,Alex,Math


In [6]:
# stuexa 保存 students 每次参加 examinations
stuexa = pd.merge(students, examinations, on='student_id').drop('key', axis=1)
stuexa

,student_id,student_name,subject_name
0,1,Alice,Math
1,1,Alice,Physics
2,1,Alice,Programming
3,1,Alice,Physics
4,1,Alice,Math
5,1,Alice,Math
6,2,Bob,Programming
7,2,Bob,Math
8,13,John,Math
9,13,John,Programming


In [7]:
# grouped 保存 student 参加某种 subject 的 examinations 的次数
grouped = stuexa.groupby(['student_id', 'subject_name']).size().reset_index()
grouped

,student_id,subject_name,0
0,1,Math,3
1,1,Physics,2
2,1,Programming,1
3,2,Math,1
4,2,Programming,1
5,13,Math,1
6,13,Physics,1
7,13,Programming,1


In [8]:
result = pd.merge(stusub, grouped, on=['student_id', 'subject_name'], how='outer')
result

,student_id,student_name,subject_name,0
0,1,Alice,Math,3.0
1,1,Alice,Physics,2.0
2,1,Alice,Programming,1.0
3,2,Bob,Math,1.0
4,2,Bob,Physics,NaN
5,2,Bob,Programming,1.0
6,13,John,Math,1.0
7,13,John,Physics,1.0
8,13,John,Programming,1.0
9,6,Alex,Math,NaN


In [9]:
result.rename(columns={0: 'attended_exams'}).fillna(0).sort_values(by=['student_id', 'subject_name'])

,student_id,student_name,subject_name,attended_exams
0,1,Alice,Math,3.0
1,1,Alice,Physics,2.0
2,1,Alice,Programming,1.0
3,2,Bob,Math,1.0
4,2,Bob,Physics,0.0
5,2,Bob,Programming,1.0
9,6,Alex,Math,0.0
10,6,Alex,Physics,0.0
11,6,Alex,Programming,0.0
6,13,John,Math,1.0


上面的解法有漏洞。

| student_id | student_name |
| ---------- | ------------ |
| 1          | Alice        |

| subject_name |
| ------------ |

| student_id | subject_name |
| ---------- | ------------ |

输出：

| student_name | student_id | subject_name | attended_exams |
| ------------ | ---------- | ------------ | -------------- |

预期结果：

| student_id | student_name | subject_name | attended_exams |
| ---------- | ------------ | ------------ | -------------- |

也就是说，在那些复杂的连接中，我没有确保 student_id 在 student_name 的前面。

In [10]:
result = result.rename(columns={0: 'attended_exams'}).fillna(0).sort_values(by=['student_id', 'subject_name'])
result[['student_id','student_name','subject_name','attended_exams']]

,student_id,student_name,subject_name,attended_exams
0,1,Alice,Math,3.0
1,1,Alice,Physics,2.0
2,1,Alice,Programming,1.0
3,2,Bob,Math,1.0
4,2,Bob,Physics,0.0
5,2,Bob,Programming,1.0
9,6,Alex,Math,0.0
10,6,Alex,Physics,0.0
11,6,Alex,Programming,0.0
6,13,John,Math,1.0
